# FurniMatch AI - Data Analytics Notebook

**Author:** FurniMatch AI Team  
**Date:** 2025-10-18  
**Purpose:** Comprehensive exploratory data analysis of furniture product dataset

## Objectives
1. Load and understand the dataset structure
2. Perform data quality assessment
3. Analyze product distributions (price, categories, brands)
4. Identify patterns and insights
5. Generate visualizations for key metrics

---

## 1. Setup and Imports

**Reasoning:** Import necessary libraries for data analysis and visualization. We use pandas for data manipulation, numpy for numerical operations, and matplotlib/seaborn for visualization.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Configure display and plotting
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 2. Load Dataset

**Reasoning:** Load the furniture dataset from CSV. We check if the file exists and handle potential loading errors gracefully.

In [ ]:
# Define data path
DATA_PATH = '../backend/data/furniture_dataset.csv'

# Load dataset
try:
    df = pd.read_csv(DATA_PATH)
    print(f"Dataset loaded successfully!")
    print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
except FileNotFoundError:
    print(f"ERROR: File not found at {DATA_PATH}")
    print("Please ensure the dataset is in the correct location.")

## 3. Initial Data Exploration

**Reasoning:** Get a first look at the data structure, column types, and sample records to understand what we're working with.

In [ ]:
# Display basic information
print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)
df.info()

print("\n" + "=" * 80)
print("FIRST 5 ROWS")
print("=" * 80)
df.head()

## 4. Data Quality Assessment

**Reasoning:** Identify missing values, duplicates, and data quality issues. This is critical for understanding data completeness and planning preprocessing steps.

In [ ]:
# Check for missing values
print("MISSING VALUES ANALYSIS")
print("=" * 80)
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
print(missing_data.to_string(index=False))

# Check for duplicates
print("\n" + "=" * 80)
print("DUPLICATE ANALYSIS")
print("=" * 80)
duplicates = df.duplicated(subset=['uniq_id'], keep=False).sum()
print(f"Duplicate products (by uniq_id): {duplicates}")
print(f"Percentage: {(duplicates/len(df)*100):.2f}%")

## 5. Price Analysis

**Reasoning:** Price is a critical factor in recommendations. We analyze price distribution, outliers, and statistics to understand the product range.

In [ ]:
# Clean price data
df['price_clean'] = df['price'].astype(str).str.replace('$', '').str.replace(',', '')
df['price_clean'] = pd.to_numeric(df['price_clean'], errors='coerce')

# Price statistics
print("PRICE STATISTICS")
print("=" * 80)
print(f"Mean Price: ${df['price_clean'].mean():.2f}")
print(f"Median Price: ${df['price_clean'].median():.2f}")
print(f"Std Dev: ${df['price_clean'].std():.2f}")
print(f"Min Price: ${df['price_clean'].min():.2f}")
print(f"Max Price: ${df['price_clean'].max():.2f}")
print(f"\nPrice Quartiles:")
print(df['price_clean'].describe())

# Visualize price distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df['price_clean'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Price ($)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Price Distribution (All Products)', fontsize=14, fontweight='bold')
axes[0].axvline(df['price_clean'].mean(), color='red', linestyle='--', label=f'Mean: ${df["price_clean"].mean():.2f}')
axes[0].axvline(df['price_clean'].median(), color='green', linestyle='--', label=f'Median: ${df["price_clean"].median():.2f}')
axes[0].legend()

# Box plot
axes[1].boxplot(df['price_clean'].dropna(), vert=True)
axes[1].set_ylabel('Price ($)', fontsize=12)
axes[1].set_title('Price Distribution (Box Plot)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Category Analysis

**Reasoning:** Understanding category distribution helps identify the most common furniture types and ensures balanced recommendations across categories.

In [ ]:
# Extract main categories
# Categories are stored as strings that may contain lists or comma-separated values
def extract_main_category(cat_str):
    if pd.isna(cat_str) or cat_str == '':
        return 'Unknown'
    # Remove brackets and quotes, take first category
    clean = str(cat_str).replace('[', '').replace(']', '').replace("'", '').replace('"', '')
    categories = [c.strip() for c in clean.split(',')]
    return categories[0] if categories else 'Unknown'

df['main_category'] = df['categories'].apply(extract_main_category)

# Category distribution
print("CATEGORY DISTRIBUTION (Top 15)")
print("=" * 80)
category_counts = df['main_category'].value_counts().head(15)
print(category_counts)

# Visualize top categories
plt.figure(figsize=(12, 6))
category_counts.plot(kind='barh', color='skyblue', edgecolor='black')
plt.xlabel('Number of Products', fontsize=12)
plt.ylabel('Category', fontsize=12)
plt.title('Top 15 Product Categories', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 7. Brand Analysis

**Reasoning:** Brand diversity affects recommendation quality. We analyze top brands and their market presence in the dataset.

In [ ]:
# Brand distribution
print("BRAND ANALYSIS (Top 20)")
print("=" * 80)
brand_counts = df['brand'].value_counts().head(20)
print(brand_counts)

print(f"\nTotal unique brands: {df['brand'].nunique():,}")
print(f"Average products per brand: {len(df) / df['brand'].nunique():.2f}")

# Visualize top brands
plt.figure(figsize=(12, 6))
brand_counts.head(15).plot(kind='bar', color='coral', edgecolor='black')
plt.xlabel('Brand', fontsize=12)
plt.ylabel('Number of Products', fontsize=12)
plt.title('Top 15 Brands by Product Count', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 8. Material and Color Analysis

**Reasoning:** Material and color are important filters in our recommendation system. Analyzing their distribution helps understand product diversity.

In [ ]:
# Material analysis
print("MATERIAL DISTRIBUTION (Top 15)")
print("=" * 80)
material_counts = df['material'].value_counts().head(15)
print(material_counts)

# Color analysis
print("\n" + "=" * 80)
print("COLOR DISTRIBUTION (Top 15)")
print("=" * 80)
color_counts = df['color'].value_counts().head(15)
print(color_counts)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Material distribution
material_counts.head(10).plot(kind='barh', ax=axes[0], color='lightgreen', edgecolor='black')
axes[0].set_xlabel('Count', fontsize=12)
axes[0].set_ylabel('Material', fontsize=12)
axes[0].set_title('Top 10 Materials', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()

# Color distribution
color_counts.head(10).plot(kind='barh', ax=axes[1], color='lightblue', edgecolor='black')
axes[1].set_xlabel('Count', fontsize=12)
axes[1].set_ylabel('Color', fontsize=12)
axes[1].set_title('Top 10 Colors', fontsize=14, fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

## 9. Price by Category Analysis

**Reasoning:** Understanding price variations across categories helps identify premium vs. budget product segments.

In [ ]:
# Price statistics by top categories
top_categories = df['main_category'].value_counts().head(10).index
category_price_stats = df[df['main_category'].isin(top_categories)].groupby('main_category')['price_clean'].agg([
    ('Count', 'count'),
    ('Mean', 'mean'),
    ('Median', 'median'),
    ('Min', 'min'),
    ('Max', 'max')
]).round(2)

print("PRICE STATISTICS BY CATEGORY (Top 10 Categories)")
print("=" * 80)
print(category_price_stats)

# Visualize average price by category
plt.figure(figsize=(12, 6))
category_price_stats.sort_values('Mean', ascending=True)['Mean'].plot(
    kind='barh', 
    color='mediumpurple', 
    edgecolor='black'
)
plt.xlabel('Average Price ($)', fontsize=12)
plt.ylabel('Category', fontsize=12)
plt.title('Average Price by Category', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 10. Data Quality Insights

**Reasoning:** Summarize key data quality metrics that will inform preprocessing decisions.

In [ ]:
# Check description completeness
desc_filled = df['description'].notna().sum()
desc_filled_pct = (desc_filled / len(df) * 100)

# Check image availability
images_filled = df['images'].notna().sum()
images_filled_pct = (images_filled / len(df) * 100)

print("DATA COMPLETENESS SUMMARY")
print("=" * 80)
print(f"Total Products: {len(df):,}")
print(f"Unique Products (by ID): {df['uniq_id'].nunique():,}")
print(f"\nField Completeness:")
print(f"  Titles: {df['title'].notna().sum():,} ({df['title'].notna().sum()/len(df)*100:.1f}%)")
print(f"  Descriptions: {desc_filled:,} ({desc_filled_pct:.1f}%)")
print(f"  Prices: {df['price_clean'].notna().sum():,} ({df['price_clean'].notna().sum()/len(df)*100:.1f}%)")
print(f"  Brands: {df['brand'].notna().sum():,} ({df['brand'].notna().sum()/len(df)*100:.1f}%)")
print(f"  Categories: {df['categories'].notna().sum():,} ({df['categories'].notna().sum()/len(df)*100:.1f}%)")
print(f"  Images: {images_filled:,} ({images_filled_pct:.1f}%)")
print(f"  Materials: {df['material'].notna().sum():,} ({df['material'].notna().sum()/len(df)*100:.1f}%)")
print(f"  Colors: {df['color'].notna().sum():,} ({df['color'].notna().sum()/len(df)*100:.1f}%)")

## 11. Key Insights and Recommendations

**Reasoning:** Summarize findings to inform model development and business decisions.

In [ ]:
print("KEY INSIGHTS")
print("=" * 80)
print("\n1. DATASET SIZE AND QUALITY")
print(f"   - Total products: {len(df):,}")
print(f"   - Data completeness: Good for core fields (title, price, brand)")
print(f"   - Description coverage: {desc_filled_pct:.1f}% (room for improvement)")

print("\n2. PRICE RANGE")
print(f"   - Average: ${df['price_clean'].mean():.2f}")
print(f"   - Range: ${df['price_clean'].min():.2f} - ${df['price_clean'].max():.2f}")
print(f"   - Recommendation: Consider price buckets for better filtering")

print("\n3. CATEGORY DISTRIBUTION")
print(f"   - Unique categories: {df['main_category'].nunique():,}")
print(f"   - Top category: {df['main_category'].value_counts().index[0]}")
print(f"   - Recommendation: Balanced across furniture types")

print("\n4. BRAND DIVERSITY")
print(f"   - Unique brands: {df['brand'].nunique():,}")
print(f"   - Top brand: {df['brand'].value_counts().index[0]}")
print(f"   - Recommendation: Good brand diversity for recommendations")

print("\n5. RECOMMENDATIONS FOR MODEL")
print("   - Use title and description as primary features (high completeness)")
print("   - Incorporate category, material, color as secondary features")
print("   - Handle missing descriptions with title-based fallback")
print("   - Price-based filtering will be effective (good coverage)")
print("   - Consider brand as a quality signal")

print("\n" + "=" * 80)
print("ANALYSIS COMPLETE")
print("=" * 80)